### This notebook contains code for training the version of the model that runs on only on the GPUs (and is incompatible with TPUs).
The conda environment required for running this notebook can be installed and activated by running the following on the command line from within this folder: 
<code>conda env create -f evolution_env_gpu_only.yml </code>  \
<code>conda activate seq</code>


In [ ]:
import argparse,pwd,os,numpy as np,h5py
from os.path import splitext,exists,dirname,join,basename
from os import makedirs
from itertools import izip
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf, sys, numpy as np, h5py, pandas as pd
from tensorflow import nn
from tensorflow.contrib import rnn
from os.path import join,dirname,basename,exists,realpath
from os import makedirs
from tensorflow.examples.tutorials.mnist import input_data
import sklearn , scipy
from sklearn.metrics import *
from scipy.stats import *
import time
import os 
from tqdm import tqdm
import datetime
from datetime import datetime


#parser = argparse.ArgumentParser(description='Input model conditions')
#parser.add_argument('-m','--model_conditions',  help='Model Conditions', required=True)
#args = vars(parser.parse_args())
args['model_conditions'] = 'pTpA_' 

model_conditions = args['model_conditions']


## Load the data matrix 
with h5py.File(join('..',model_conditions,'expression.h5'), 'r') as hf: #replace with path to your data
    expressions = hf['expression'][:]
#expressions.shape

## Load the sequences 
with h5py.File(join('..',model_conditions,'onehot_sequences_bool.h5'), 'r') as hf:#replace with path to your data
    onehot_sequences = hf['onehot_sequences_bool'][:]
#onehot_sequences.shape

## Load the rc sequences 
with h5py.File(join('..',model_conditions,'rc_onehot_sequences_bool.h5'), 'r') as hf:#replace with path to your data
    rc_onehot_sequences = hf['rc_onehot_sequences_bool'][:]


    
    


## training, validation and test data split
## Shuffling 
randomize  =  np.random.permutation(len(onehot_sequences))
onehot_sequences = onehot_sequences[randomize,:]
rc_onehot_sequences = rc_onehot_sequences[randomize,:]
expressions = expressions[randomize]
expressions = np.reshape(expressions , [-1,1])

total_seqs = len(onehot_sequences)
_trX = onehot_sequences[int(total_seqs/10):]
_trX_rc = rc_onehot_sequences[int(total_seqs/10):]
_trY = expressions[int(total_seqs/10):]

_vaX = onehot_sequences[0:int(total_seqs/10)]
_vaX_rc = rc_onehot_sequences[0:int(total_seqs/10)]
_vaY = expressions[0:int(total_seqs/10)]




_trX.shape , _trY.shape , _vaX.shape , _vaY.shape  





##MODEL FILE SAVING ADDRESSES AND MINIBATCH SIZES
_batch_size = 1024
_hyper_train_size = 2000
#_valid_size = 1024
_hidden = 256
_epochs = 15
_best_model_file = join('..',model_conditions,'models','best_model.ckpt')
_best_model_file_hyper = join('..',model_conditions,'models','hyper_search', 'best_model.ckpt')
for _file  in [_best_model_file, _best_model_file_hyper]:
    if not exists(dirname(_file)):
        makedirs(dirname(_file))

        











####################################################################
####################################################################
### MODEL ARCHITECTURE
####################################################################
####################################################################
def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride=1):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='SAME')

def max_pool(x, stride=2, filter_size=2):
    return tf.nn.max_pool(x, ksize=[1, 1, 2, 1],
                        strides=[1, 1, 2, 1], padding='SAME')

def cross_entropy(y, y_real):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels = y_real))

def build_two_fc_layers(x_inp, Ws, bs):
    h_fc1 = tf.nn.relu(tf.matmul(x_inp, Ws[0]) + bs[0])
    return tf.matmul(h_fc1, Ws[1]) + bs[1]



            
            


def cnn_model(X, hyper_params , scope):

    with tf.variable_scope(scope) : 
        global _hidden 
        conv1_filter_dim1 = 30
        conv1_filter_dim2 = 4
        conv1_depth = _hidden

        conv2_filter_dim1 = 30
        conv2_filter_dim2 = 1
        conv2_depth = _hidden








        W_conv1 = weight_variable([1,conv1_filter_dim1,conv1_filter_dim2,conv1_depth])
        conv1 = conv2d(X, W_conv1)    
        conv1 = tf.nn.bias_add(conv1, bias_variable([conv1_depth]))
        conv1 = tf.nn.relu(conv1)
        l_conv = conv1
        
        W_conv2 = weight_variable([conv2_filter_dim1,conv2_filter_dim2,conv1_depth, conv2_depth])
        conv2 = conv2d(conv1,W_conv2 )
        conv2 = tf.nn.bias_add(conv2, bias_variable([conv2_depth]))
        conv2 = tf.nn.relu(conv2)

        
        regularization_term = hyper_params['l2']* tf.reduce_mean(tf.abs(W_conv1)) + hyper_params['l2']* tf.reduce_mean(tf.abs(W_conv2)) 
        
        cnn_model_output = conv2

    return cnn_model_output , regularization_term 






def training(trX, trX_rc, trY, valX, valX_rc, valY, hyper_params, epochs, batch_size, best_model_file): 

    
    tf.reset_default_graph()
    
    global _hidden 

    lstm_num_hidden = _hidden
    fc_num_hidden = _hidden
    num_classes = 1
    num_bins = 256
    
    conv3_filter_dim1 = 30
    conv3_filter_dim2 = 1
    conv3_depth = _hidden
    
    conv4_filter_dim1 = 30
    conv4_filter_dim2 = 1
    conv4_depth = _hidden
        # Input and output

    X = tf.placeholder("float", [None, 1, 110, 4] )
    X_rc = tf.placeholder("float", [None, 1, 110, 4] )
    Y = tf.placeholder("float", [None,1] )
    dropout_keep_probability = tf.placeholder_with_default(1.0, shape=())


    #f is forward sequence 
    output_f , regularization_term_f =  cnn_model(X, {'dropout_keep':hyper_params['dropout_keep'],'l2':hyper_params['l2']} , "f")

    #rc is reverse complement of that sequence
    output_rc , regularization_term_rc =  cnn_model(X_rc, {'dropout_keep':hyper_params['dropout_keep'],'l2':hyper_params['l2']} , "rc")
    
    
    ### CONCATENATE output_f and output_rc
    concatenated_f_rc = tf.concat([output_f , output_rc], -1)
    ###
    
    W_conv3 = weight_variable([conv3_filter_dim1,conv3_filter_dim2,2*_hidden,conv3_depth])
    conv3 = conv2d(concatenated_f_rc,W_conv3 )
    conv3 = tf.nn.bias_add(conv3, bias_variable([conv3_depth]))
    conv3 = tf.nn.relu(conv3)

    W_conv4 = weight_variable([conv4_filter_dim1,conv4_filter_dim2,conv3_depth,conv4_depth])
    conv4 = conv2d(conv3,W_conv4 )
    conv4 = tf.nn.bias_add(conv4, bias_variable([conv4_depth]))
    conv4 = tf.nn.relu(conv4)


    conv_feat_map_x = 110   
    conv_feat_map_y =  1   
    h_conv_flat = tf.reshape(conv4, [-1, conv_feat_map_x * conv_feat_map_y * lstm_num_hidden])
    #FC-1
    
    W_fc1 = weight_variable([conv_feat_map_x * conv_feat_map_y * lstm_num_hidden , fc_num_hidden])
    b_fc1 = bias_variable([fc_num_hidden])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv_flat, W_fc1) + b_fc1)
    #Dropout for FC-1
    h_fc1 = tf.nn.dropout(h_fc1, dropout_keep_probability)

    
    #FC-2
    W_fc2 = weight_variable([fc_num_hidden , num_bins])
    b_fc2 = bias_variable([num_bins])
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
    #Dropout for FC-2
    h_fc2 = tf.nn.dropout(h_fc2, dropout_keep_probability)


    #FC-3
    W_fc3 = weight_variable([num_bins, num_classes])
    b_fc3 = bias_variable([num_classes])
    h_fc3 = tf.matmul(h_fc2, W_fc3) + b_fc3 

    regularization_term = hyper_params['l2']* tf.reduce_mean(tf.abs(W_fc3)) + hyper_params['l2']* tf.reduce_mean(tf.abs(W_fc2)) + hyper_params['l2']* tf.reduce_mean(tf.abs(W_fc1)) + hyper_params['l2']* tf.reduce_mean(tf.abs(W_conv3))+ regularization_term_f + regularization_term_rc +hyper_params['l2']* tf.reduce_mean(tf.abs(W_conv4))



    with tf.variable_scope("out") :
        output = h_fc3
        model_output = tf.identity(output, name="model_output")

        ##########

        loss = tf.losses.mean_squared_error( Y , model_output ) + regularization_term
        cost = loss 
        model_cost = tf.identity(cost, name="model_cost")
        ##########
        pcc = tf.contrib.metrics.streaming_pearson_correlation(model_output,Y)
        model_pcc = tf.identity(pcc, name="model_pcc")
        ##########
        mse = tf.losses.mean_squared_error( Y , model_output )
        model_mse = tf.identity(mse, name="model_mse")
        ##########
        total_error = tf.reduce_sum(tf.square(tf.subtract(Y, tf.reduce_mean(Y))))
        unexplained_error = tf.reduce_sum(tf.square(tf.subtract(Y, model_output)))
        R_squared = tf.subtract(tf.constant(
    1,
    dtype=tf.float32), tf.div(unexplained_error, total_error))
        model_R_squared = tf.identity(R_squared, name="model_R_squared")

        ##########

        


    tf.summary.scalar("cost", model_cost)
    tf.summary.scalar("pcc", model_pcc[0])
    tf.summary.scalar("mse", model_mse)
    tf.summary.scalar("R_squared", R_squared)

    summary_op = tf.summary.merge_all()
    
    train_op = tf.train.AdamOptimizer(hyper_params['lr']).minimize(cost)


    start = time.time()
    best_cost = float("inf") 
    best_r2 = float(0) 

    
    batches_per_epoch = int(len(trX)/batch_size)
    num_steps = int(epochs * batches_per_epoch)

    sess = tf.Session()
    init=tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    #init = tf.global_variables_initializer()
    sess.run(init)
    
    #clear the logs directory

    now = datetime.now()
    writer = tf.summary.FileWriter(join('..',model_conditions,'logs' , now.strftime("%Y%m%d-%H%M%S") ), sess.graph)

    print 'Initializing variables...'

    epoch_loss = 0
    epoch_pcc = 0
    epoch_mse = 0
    epoch_R_squared = 0

    
   
    
    
    
    for step in tqdm(xrange(num_steps)):
    
    
    
        offset = (step * batch_size) % (trX.shape[0] - batch_size)
        batch_x = trX[offset:(offset + batch_size), :]
        batch_x_rc = trX_rc[offset:(offset + batch_size), :]
        batch_y = trY[offset:(offset + batch_size)]

        
        feed_dict = {X: batch_x, X_rc: batch_x_rc ,  Y: batch_y , dropout_keep_probability : hyper_params['dropout_keep'] }
        
        _, batch_loss , batch_pcc , batch_mse, batch_R_squared , summary = sess.run([train_op, cost , pcc , mse, R_squared , summary_op], feed_dict=feed_dict)
        
        batch_R_squared = batch_pcc[0]**2
        epoch_loss += batch_loss
        epoch_pcc += batch_pcc[0]
        epoch_mse += batch_mse        
        epoch_R_squared += batch_R_squared    
        
        writer.add_summary(summary, step)

        
        
        if ( (step % batches_per_epoch == 0) and step/batches_per_epoch!=0):

            epoch_loss /= batches_per_epoch
            epoch_pcc /= batches_per_epoch
            epoch_mse /= batches_per_epoch
            epoch_R_squared /= batches_per_epoch
            
            print ''
            print ''
            print ''
            print ''
            print 'Training - Avg batch loss at epoch %d: %f' % (step/batches_per_epoch, epoch_loss)
            print 'Training - PCC : %f' % epoch_pcc
            print 'Training - MSE : %f' % epoch_mse
            print 'Training - R_squared : %f' % epoch_pcc**2
            
            epoch_loss = 0
            epoch_pcc = 0
            epoch_mse = 0
            epoch_R_squared = 0  
            
            #Randomized validation subset start
            randomize  =  np.random.permutation(len(valX))
            vaX = valX[randomize,:]
            vaX_rc = valX_rc[randomize,:]
            vaY = valY[randomize,:]
            
            #valX = vaX[0:valid_size,:] 
            #valX_rc = vaX_rc[0:valid_size,:] 
            #valY = vaY[0:valid_size,:]            
            
            #with tf.device('/cpu:0'):
            #validation_cost , validation_acc , summary = sess.run([cost , accuracy , summary_op], feed_dict={X: valX , X_rc: valX_rc, Y: valY})
            
            #### teX_output contains TESTED SEQUENCES FOR VALIDATION SET
            va_batch_size = 1024
            (q,r) = divmod(vaX.shape[0] , va_batch_size)
            i=0
            vaX_output = []
            while(i <= q ) :
                if(i< q  ) :
                    temp_result_step1=sess.run([model_output], feed_dict={X: vaX[va_batch_size*i:va_batch_size*i+va_batch_size,:], X_rc: vaX_rc[va_batch_size*i:va_batch_size*i+va_batch_size,:]  ,Y: vaY[va_batch_size*i:va_batch_size*i+va_batch_size,:]}) 
                    temp_result_step2=[float(x) for x in temp_result_step1[0]]
                    #print temp_result_step2

                    vaX_output = vaX_output + temp_result_step2
                    i = i+1

                elif (i==q) :
                    temp_result_step1 = sess.run([model_output], feed_dict={X: vaX[va_batch_size*i:,:], X_rc: vaX_rc[va_batch_size*i:,:]  ,Y: vaY[va_batch_size*i:,:]})
                    temp_result_step2=[float(x) for x in temp_result_step1[0]]
                    #print "here"
                    vaX_output = vaX_output + temp_result_step2
                    i = i+1

            #### RETURN TESTED SEQUENCES FOR VALIDATION SET
            vaY = [float(x) for x in vaY]
            validation_mse = sklearn.metrics.mean_squared_error(vaY , vaX_output )

            validation_pcc = scipy.stats.pearsonr(vaY , vaX_output )
            validation_r2  = validation_pcc[0]**2
        
        
        
            #for tensorboard
            print ''
            print 'Full Validation Set - MSE : %f' % validation_mse
            print 'Full Validation Set - PCC : %f' % validation_pcc[0]
            print 'Full Validation Set - R_squared : %f' % validation_r2

            if(best_r2 < validation_r2) :
                #
                
                #SAVER 
                saver = tf.train.Saver()               
                saver.save(sess, "%s"%best_model_file )

                #
                best_loss = validation_mse
                best_cost = validation_mse
                best_r2 = validation_r2

                
    print "Training time: ", time.time() - start
    

    return best_r2


# Training
best_dropout = 0.8  
best_l2_coef = 0.0001 
best_lr = 0.0005     


print '\n', training(_trX,_trX_rc, _trY, _vaX,_vaX_rc, _vaY, \
                    {'dropout_keep':best_dropout,'l2':best_l2_coef, 'lr':best_lr},\
        _epochs, _batch_size , _best_model_file)
